In [ ]:
import pandas as pd
import re

GROUND_TRUTH_MAIL = ''
file_path = 'outputs/excel/group5_results.xlsx'  # TODO: update
txt_output_path = 'outputs/txt/group5.txt'  # TODO: update

task_agreements = [59.81, 72.16, 59.62, 62.27, 55.54] # TODO: update
task_ids = [127902439, 127902443, 127902456, 127902467, 127902480] # TODO: update
group = 'group5' # TODO: update
nan_position = False # TODO: update


# Read the Excel file
df = pd.read_excel(file_path, sheet_name='Sheet1')

# Filter out the ground truth mail
number_of_tests = df.loc[df['mail'] == GROUND_TRUTH_MAIL, 'number_of_tasks_completed'].values[0]
df = df[df['mail'] != GROUND_TRUTH_MAIL]

def calculate_f1(tp, fp, fn):
    precision = tp / (tp + fp) if (tp + fp) != 0 else 0
    recall = tp / (tp + fn) if (tp + fn) != 0 else 0
    return 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

def calculate_kappa(tp, fp, fn, tn):
    kappa_above = 2*(tp*tn - fn*fp)
    kappa_below = (tp+fp)*(fp+tn)+(tp+fn)*(fn+tn)
    kappa = kappa_above / kappa_below if kappa_below > 0 else 0
    return kappa


def calculate_exact_match(intersection, difference):
    return intersection / (intersection + difference) if (intersection + difference) != 0 else 0

def generate_results(df, task_prefix):
    # Overall results for the task
    eng_term_detection_f1 = calculate_f1(df[f'{task_prefix}_english_term_detection_tp'].sum(), 
                                         df[f'{task_prefix}_english_term_detection_fp'].sum(), 
                                         df[f'{task_prefix}_english_term_detection_fn'].sum())
    
    tr_term_detection_f1 = calculate_f1(df[f'{task_prefix}_turkish_term_detection_tp'].sum(), 
                                        df[f'{task_prefix}_turkish_term_detection_fp'].sum(), 
                                        df[f'{task_prefix}_turkish_term_detection_fn'].sum())
    
    eng_term_detection_kappa = calculate_kappa(df[f'{task_prefix}_english_term_detection_tp'].sum(),
                                                df[f'{task_prefix}_english_term_detection_fp'].sum(),
                                                df[f'{task_prefix}_english_term_detection_fn'].sum(),
                                                df[f'{task_prefix}_english_term_detection_tn'].sum())
    
    tr_term_detection_kappa = calculate_kappa(df[f'{task_prefix}_turkish_term_detection_tp'].sum(),
                                                  df[f'{task_prefix}_turkish_term_detection_fp'].sum(),
                                                  df[f'{task_prefix}_turkish_term_detection_fn'].sum(),
                                                  df[f'{task_prefix}_turkish_term_detection_tn'].sum())
    
    
    
    exact_match_turkish_labels = calculate_exact_match(df[f'{task_prefix}_turkish_labels_intersection_num'].sum(), 
                                                       df[f'{task_prefix}_turkish_labels_difference_num'].sum())
    
    exact_match_turkish_corrections = calculate_exact_match(df[f'{task_prefix}_turkish_corrections_intersection_num'].sum(), 
                                                            df[f'{task_prefix}_turkish_corrections_difference_num'].sum())
    
    exact_match_english_term_linking = calculate_exact_match(df[f'{task_prefix}_english_term_linking_intersection_num'].sum(), 
                                                             df[f'{task_prefix}_english_term_linking_difference_num'].sum())
    
    # Collect overall results for the specified task
    overall_results = {
        "task": task_prefix,
        "eng_term_detection_f1": round(eng_term_detection_f1, 3),
        "tr_term_detection_f1": round(tr_term_detection_f1, 3),
        "eng_term_detection_kappa": round(eng_term_detection_kappa, 3),
        "tr_term_detection_kappa": round(tr_term_detection_kappa, 3),
        "exact_match_turkish_labels": round(exact_match_turkish_labels, 3),
        "exact_match_turkish_corrections": round(exact_match_turkish_corrections, 3),
        "exact_match_english_term_linking": round(exact_match_english_term_linking, 3),
    }
    
    # Collect results for each annotator
    annotator_results = []
    for _, row in df.iterrows():
        eng_term_detection_f1 = calculate_f1(row[f'{task_prefix}_english_term_detection_tp'], 
                                             row[f'{task_prefix}_english_term_detection_fp'], 
                                             row[f'{task_prefix}_english_term_detection_fn'])
        
        tr_term_detection_f1 = calculate_f1(row[f'{task_prefix}_turkish_term_detection_tp'], 
                                            row[f'{task_prefix}_turkish_term_detection_fp'], 
                                            row[f'{task_prefix}_turkish_term_detection_fn'])
        
        eng_term_detection_kappa = calculate_kappa(row[f'{task_prefix}_english_term_detection_tp'],
                                                row[f'{task_prefix}_english_term_detection_fp'],
                                                row[f'{task_prefix}_english_term_detection_fn'],
                                                row[f'{task_prefix}_english_term_detection_tn'])
        
        tr_term_detection_kappa = calculate_kappa(row[f'{task_prefix}_turkish_term_detection_tp'],
                                                    row[f'{task_prefix}_turkish_term_detection_fp'],
                                                    row[f'{task_prefix}_turkish_term_detection_fn'],
                                                    row[f'{task_prefix}_turkish_term_detection_tn'])
        
        
        exact_match_turkish_labels = calculate_exact_match(row[f'{task_prefix}_turkish_labels_intersection_num'], 
                                                           row[f'{task_prefix}_turkish_labels_difference_num'])
        
        exact_match_turkish_corrections = calculate_exact_match(row[f'{task_prefix}_turkish_corrections_intersection_num'], 
                                                                row[f'{task_prefix}_turkish_corrections_difference_num'])
        
        exact_match_english_term_linking = calculate_exact_match(row[f'{task_prefix}_english_term_linking_intersection_num'], 
                                                                 row[f'{task_prefix}_english_term_linking_difference_num'])
        
        annotator_results.append({
            "annotator": row['mail'],
            "eng_term_detection_f1": round(eng_term_detection_f1, 3),
            "tr_term_detection_f1": round(tr_term_detection_f1, 3),
            "eng_term_detection_kappa": round(eng_term_detection_kappa, 3),
            "tr_term_detection_kappa": round(tr_term_detection_kappa, 3),
            "exact_match_turkish_labels": round(exact_match_turkish_labels, 3),
            "exact_match_turkish_corrections": round(exact_match_turkish_corrections, 3),
            "exact_match_english_term_linking": round(exact_match_english_term_linking, 3),
        })

    return overall_results, annotator_results

# Identify all unique task prefixes
task_columns = df.columns[df.columns.str.contains(r'^task_\d+')]
task_prefixes = {re.match(r'^(task_\d+)', col).group(1) for col in task_columns}

# Cumulative Results
cumulative_eng_term_detection_f1 = calculate_f1(df['cumulative_english_term_detection_tp'].sum(), df['cumulative_english_term_detection_fp'].sum(), df['cumulative_english_term_detection_fn'].sum())
cumulative_tr_term_detection_f1 = calculate_f1(df['cumulative_turkish_term_detection_tp'].sum(), df['cumulative_turkish_term_detection_fp'].sum(), df['cumulative_turkish_term_detection_fn'].sum())
exact_match_turkish_labels = calculate_exact_match(df['cumulative_turkish_labels_intersection_num'].sum(), df['cumulative_turkish_labels_difference_num'].sum())
exact_match_turkish_corrections = calculate_exact_match(df['cumulative_turkish_corrections_intersection_num'].sum(), df['cumulative_turkish_corrections_difference_num'].sum())
exact_match_english_term_linking = calculate_exact_match(df['cumulative_english_term_linking_intersection_num'].sum(), df['cumulative_english_term_linking_difference_num'].sum())

cumulative_eng_term_detection_kappa = calculate_kappa(df['cumulative_english_term_detection_tp'].sum(), df['cumulative_english_term_detection_fp'].sum(), df['cumulative_english_term_detection_fn'].sum(), df['cumulative_english_term_detection_tn'].sum())

cumulative_tr_term_detection_kappa = calculate_kappa(df['cumulative_turkish_term_detection_tp'].sum(), df['cumulative_turkish_term_detection_fp'].sum(), df['cumulative_turkish_term_detection_fn'].sum(), df['cumulative_turkish_term_detection_tn'].sum())


cumulative_results = {
    "eng_term_detection_f1": round(cumulative_eng_term_detection_f1, 3),
    "tr_term_detection_f1": round(cumulative_tr_term_detection_f1, 3),
    "eng_term_detection_kappa": round(cumulative_eng_term_detection_kappa, 3),
    "tr_term_detection_kappa": round(cumulative_tr_term_detection_kappa, 3),
    "exact_match_turkish_labels": round(exact_match_turkish_labels, 3),
    "exact_match_turkish_corrections": round(exact_match_turkish_corrections, 3),
    "exact_match_english_term_linking": round(exact_match_english_term_linking, 3),
}

# Collect cumulative results for each annotator
cumulative_annotator_results = []
for _, row in df.iterrows():
    cumulative_eng_term_detection_f1 = calculate_f1(row['cumulative_english_term_detection_tp'], row['cumulative_english_term_detection_fp'], row['cumulative_english_term_detection_fn'])
    cumulative_tr_term_detection_f1 = calculate_f1(row['cumulative_turkish_term_detection_tp'], row['cumulative_turkish_term_detection_fp'], row['cumulative_turkish_term_detection_fn'])
    cumulative_eng_term_detection_kappa = calculate_kappa(row['cumulative_english_term_detection_tp'], row['cumulative_english_term_detection_fp'], row['cumulative_english_term_detection_fn'], row['cumulative_english_term_detection_tn'])
    cumulative_tr_term_detection_kappa = calculate_kappa(row['cumulative_turkish_term_detection_tp'], row['cumulative_turkish_term_detection_fp'], row['cumulative_turkish_term_detection_fn'], row['cumulative_turkish_term_detection_tn'])
    exact_match_turkish_labels = calculate_exact_match(row['cumulative_turkish_labels_intersection_num'], row['cumulative_turkish_labels_difference_num'])
    exact_match_turkish_corrections = calculate_exact_match(row['cumulative_turkish_corrections_intersection_num'], row['cumulative_turkish_corrections_difference_num'])
    exact_match_english_term_linking = calculate_exact_match(row['cumulative_english_term_linking_intersection_num'], row['cumulative_english_term_linking_difference_num'])
    
    cumulative_annotator_results.append({
        "annotator": row['mail'],
        "eng_term_detection_f1": round(cumulative_eng_term_detection_f1, 3),
        "tr_term_detection_f1": round(cumulative_tr_term_detection_f1, 3),
        "eng_term_detection_kappa": round(cumulative_eng_term_detection_kappa, 3),
        "tr_term_detection_kappa": round(cumulative_tr_term_detection_kappa, 3),
        "exact_match_turkish_labels": round(exact_match_turkish_labels, 3),
        "exact_match_turkish_corrections": round(exact_match_turkish_corrections, 3),
        "exact_match_english_term_linking": round(exact_match_english_term_linking, 3),
    })

# Generate results for all detected tasks
task_results = {}
for task_prefix in sorted(task_prefixes):
    overall_results, annotator_results = generate_results(df, task_prefix)
    task_results[task_prefix] = {
        "overall": overall_results,
        "annotators": annotator_results
    }
    
# Generate the summary text based on the results

summary_text = ""

# Add Cumulative Results (Overall)
summary_text += "CUMULATIVE RESULTS (Overall) For "
summary_text += group + ":\n"
summary_text += f"- Cumulative English Term Detection F1: {cumulative_results['eng_term_detection_f1']}\n"
summary_text += f"- Cumulative Turkish Term Detection F1: {cumulative_results['tr_term_detection_f1']}\n"
summary_text += f"- Cumulative English Term Detection Kappa: {cumulative_results['eng_term_detection_kappa']}\n"
summary_text += f"- Cumulative Turkish Term Detection Kappa: {cumulative_results['tr_term_detection_kappa']}\n"
summary_text += f"- Cumulative Turkish Labels Exact Match: {cumulative_results['exact_match_turkish_labels']}\n"
summary_text += f"- Cumulative Turkish Corrections Exact Match: {cumulative_results['exact_match_turkish_corrections']}\n"
summary_text += f"- Cumulative English Term Linking Exact Match: {cumulative_results['exact_match_english_term_linking']}\n\n"

print(summary_text)

# Add Cumulative Results for Each Annotator
summary_text += "CUMULATIVE RESULTS FOR EACH ANNOTATOR:\n"
for annotator_result in cumulative_annotator_results:
    summary_text += f"\tAnnotator: {annotator_result['annotator']}\n"
    summary_text += f"\t  - English Term Detection F1: {annotator_result['eng_term_detection_f1']}\n"
    summary_text += f"\t  - Turkish Term Detection F1: {annotator_result['tr_term_detection_f1']}\n"
    summary_text += f"\t  - English Term Detection Kappa: {annotator_result['eng_term_detection_kappa']}\n"
    summary_text += f"\t  - Turkish Term Detection Kappa: {annotator_result['tr_term_detection_kappa']}\n"
    summary_text += f"\t  - Turkish Labels Exact Match: {annotator_result['exact_match_turkish_labels']}\n"
    summary_text += f"\t  - Turkish Corrections Exact Match: {annotator_result['exact_match_turkish_corrections']}\n"
    summary_text += f"\t  - English Term Linking Exact Match: {annotator_result['exact_match_english_term_linking']}\n\n"
    
    

# Add Task-Specific Results
for task, results in task_results.items():
    summary_text += f"{task.upper()} RESULTS (Overall):\n"
    summary_text += f"- English Term Detection F1: {results['overall']['eng_term_detection_f1']}\n"
    summary_text += f"- Turkish Term Detection F1: {results['overall']['tr_term_detection_f1']}\n"
    summary_text += f"- English Term Detection Kappa: {results['overall']['eng_term_detection_kappa']}\n"
    summary_text += f"- Turkish Term Detection Kappa: {results['overall']['tr_term_detection_kappa']}\n"
    summary_text += f"- Turkish Labels Exact Match: {results['overall']['exact_match_turkish_labels']}\n"
    summary_text += f"- Turkish Corrections Exact Match: {results['overall']['exact_match_turkish_corrections']}\n"
    summary_text += f"- English Term Linking Exact Match: {results['overall']['exact_match_english_term_linking']}\n\n"
    
    summary_text += f"{task.upper()} RESULTS FOR EACH ANNOTATOR:\n"
    for annotator_result in results['annotators']:
        summary_text += f"\tAnnotator: {annotator_result['annotator']}\n"
        summary_text += f"\t  - English Term Detection F1: {annotator_result['eng_term_detection_f1']}\n"
        summary_text += f"\t  - Turkish Term Detection F1: {annotator_result['tr_term_detection_f1']}\n"
        summary_text += f"\t  - English Term Detection Kappa: {annotator_result['eng_term_detection_kappa']}\n"
        summary_text += f"\t  - Turkish Term Detection Kappa: {annotator_result['tr_term_detection_kappa']}\n"
        summary_text += f"\t  - Turkish Labels Exact Match: {annotator_result['exact_match_turkish_labels']}\n"
        summary_text += f"\t  - Turkish Corrections Exact Match: {annotator_result['exact_match_turkish_corrections']}\n"
        summary_text += f"\t  - English Term Linking Exact Match: {annotator_result['exact_match_english_term_linking']}\n\n"

# save the summary text to a file
with open(txt_output_path, 'w') as f:
    f.write(summary_text)

In [ ]:
# store overall task results in a dataframe, add task_agreements and task_ids, and group too
overall_task_results = []
count = 0
for task, results in task_results.items():
    overall_task_results.append({
        "task": task,
        "english_term_detection_f1": results['overall']['eng_term_detection_f1'],
        "turkish_term_detection_f1": results['overall']['tr_term_detection_f1'],
        "english_term_detection_kappa": results['overall']['eng_term_detection_kappa'],
        "turkish_term_detection_kappa": results['overall']['tr_term_detection_kappa'],
        "exact_match_turkish_labels": results['overall']['exact_match_turkish_labels'],
        "exact_match_turkish_corrections": results['overall']['exact_match_turkish_corrections'],
        "exact_match_english_term_linking": results['overall']['exact_match_english_term_linking'],
        "task_agreement": task_agreements[count],
        "task_id": task_ids[count],
        "group": group, 
        "nan_position": nan_position
    })
    count += 1

overall_task_results_df = pd.DataFrame(overall_task_results)
# save the dataframe to a xlsx file with group name and nan_position
overall_task_results_df.to_excel(f"outputs/summary/{group}.xlsx", index=False)

In [ ]:
overall_task_results_df